$
Code d'analyse de tracking. Il permet de récupérer les fichiers issus du tracking et d'en tirer les trajectoires pertinentes et d'en faire l'étude statistique. 

In [ ]:
%reset

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 12:46:56 2023.

@author: souchaud
"""
# %%
import os
import time
import math
import pandas as pd
import numpy as np
from skimage import io
from PIL import Image
from scipy.stats import norm
from scipy import stats
from scipy.signal import find_peaks
from scipy.optimize import curve_fit, minimize_scalar
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
import matplotlib.gridspec as gridspec
import trackpy as tp
import functions_analyze as lib
import warnings
import importlib
from collections import defaultdict
from colorama import init, Fore, Style
from typing import List, Optional, Union, Any, Dict, Tuple
importlib.reload(lib)

init(autoreset=True)
# from matplotlib.cm import ScalarMappable
# import pdb; pdb.set_trace()
# warnings.simplefilter("always")  # This will always display warnings
# warnings.simplefilter('error', RuntimeWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)

Initialisation des variables et constantes de travail.

In [ ]:
# set initial time
INITIAL_TIME = time.time()

# experiment parameters
TIME_FRAME = 15 # 15  # 75
SIZE_PIX = 0.637# 1.2773  # 1.634  # 4.902
FPS = 1/TIME_FRAME

# File to study
file_name = 'filtered_final' # 'features'  # 'filtered'Any
# number of frame kept
N_FRAME = 2
N_FRAME_MIN_STUDY = 100  # Nombre minimal de frame sur laquelle la cellule doit être suivie

# nber hours of stydy:
LONG_TIME = False

# Study parameters
ROLLING_MEAN = False
PIXELISATION = False
TIME_FRAME_STUDY = False
DRIFT = False

# plot parameters
IMG_TYPE = 'jpg'
ALPHA = 0.5
LINEWIDTH = 0.1
COLOR_SUP = 'blue'
COLOR_INF = 'red'
color_sup_inf = (COLOR_SUP, COLOR_INF)

# % de présences de la particules sur le total de frame étudiées

FRAME_PARTICULE = 1

# ##########
# % de présences des courbes dans les frames

FRAME_PARTICULE = 0.8

Définition des path et dossiers de travails / enregistrements.

In [ ]:
# ##########################   GENERAL PATH   #################################
GENERAL_PATH = '/Users/souchaud/Desktop/Analyses/'
GENERAL_PATH_PICTURES = '/Users/souchaud/Desktop/A_analyser/'
# GENERAL_PATH_PICTURES = '/Volumes/Labo_Alex_Mac/A_analyser/'

In [ ]:
############# CONDITION SIMPLE ################
# CONDITION_simple = 'CytoOne_SorC'
# CONDITION_simple = 'NonT_SorC'
CONDITION_simple = 'CytoOne_HL5_10x'
# CONDITION_simple = 'CytoOne_HL5'

############### CONDITION ################
CONDITION = f'{CONDITION_simple}_new_param' # _longtime_new_param'
# CONDITION = 'ASMOT035_fiji'
###########################################

In [ ]:
# list of pathway to the experiments
PATHWAY_EXPERIMENT = []

if len(PATHWAY_EXPERIMENT) == 0:
    PATHWAY_EXPERIMENT = [f for f in os.listdir(GENERAL_PATH + CONDITION) if
                          os.path.isdir(os.path.join(GENERAL_PATH + CONDITION, f))]
# ##########################   Path Exp final  ###############################

PATHWAY_EXPERIMENT = [f'{GENERAL_PATH}{CONDITION}/' +
                      elem + '/mosaic/' for elem in PATHWAY_EXPERIMENT]

# ##########################   Path to Save pic  ##############################

path_save_pic = f'{GENERAL_PATH}résultats_{CONDITION}_ALL_OK_x5_15s/'

# création d'un dossier spécific d'enregistrement.
if not os.path.exists(path_save_pic):
    os.mkdir(path_save_pic)
os.chdir(path_save_pic)

Lecture des données expériementales

In [ ]:
importlib.reload(lib)
DATA = lib.read_hdf5_all(pathway_experiment=PATHWAY_EXPERIMENT, name_file=file_name,
                         nbr_frame_min=N_FRAME_MIN_STUDY, condition=CONDITION, drift=DRIFT,
                         search_range=20, memory=5)
# Trier le DataFrame par la colonne "frame"
DATA = DATA.sort_values(by='frame')

In [ ]:
DATA['experiment'].unique()

On décide de travailler que sur un certain nombre de frame. Ici je décide de travailler sur les 240 première frames. 
Donc la cellules doit être suivi sur N_MIN_STUDY sur les 240 premières frames. 

In [ ]:
print("Nombre de particule avant tri: ", DATA['particle'].nunique())
DATA = DATA[DATA['frame'] < 340]

filter_data = DATA.groupby('particle').filter(lambda x: len(x) >= N_FRAME_MIN_STUDY)
print("Nombre de particule après le premier tri sur le temps suivi des cellules: ", filter_data['particle'].nunique())
DATA = filter_data

In [ ]:
DATA['time (min)'] = DATA['frame']*TIME_FRAME/60
DATA = lib.vit_instant_new(traj=DATA, lag_time=TIME_FRAME, pix_size=SIZE_PIX, triage=1)

In [ ]:
DATA = DATA.query(f'frame % {N_FRAME} == 0')
# DATA['frame'] = pd.factorize(DATA['frame'])[0]
# FPS = FPS/N_FRAME
DATA

In [ ]:
# Supposons que DATA est votre DataFrame et qu'il contient une colonne 'manip' pour identifier chaque manipulation
manips = DATA['experiment'].unique()
num_manips = len(manips)

# Créer une figure pour accueillir tous les subplots
fig = plt.figure(figsize=(16, 6*num_manips))

# Créer un gridspec pour mieux organiser les subplots
gs = gridspec.GridSpec(num_manips, 3, fig)

colors = ['skyblue', 'lightgreen', 'salmon']  # Couleurs pour les différents types de graphiques

for i, manip in enumerate(manips):
    data_manip = DATA[DATA['experiment'] == manip]
    mass_means = data_manip.groupby('particle')['mass'].mean()
    size_means = data_manip.groupby('particle')['size'].mean()

    # Masse moyenne
    ax1 = fig.add_subplot(gs[i, 0])
    ax1.hist(mass_means, bins=100, color=colors[0], density=True)
    ax1.set_title(f"Mean mass of particles for {manip}")
    ax1.set_xlabel("Mean mass")
    ax1.set_ylabel("Density")

    # Taille moyenne
    ax2 = fig.add_subplot(gs[i, 1])
    ax2.hist(size_means, bins=100, color=colors[1], density=True)
    ax2.set_title(f"Mean size of particles for {manip}")
    ax2.set_xlabel("Mean size")
    ax2.set_ylabel("Density")

    # Mass vs. Size au frame 0
    filtered_data = data_manip[data_manip['frame'] == 0]
    ax3 = fig.add_subplot(gs[i, 2])
    tp.mass_size(filtered_data, ax=ax3)
    ax3.set_title(f"Mass vs. Size at frame 0 for {manip}")

# Ajuster l'espacement entre les subplots
plt.tight_layout()

# Afficher la figure
plt.show()


In [ ]:
# # In[Filter on mass]
# if 'mass' in DATA.columns:
#     mask = DATA.groupby('particle')['mass'].transform('mean') >= 1000
#     DATA = DATA[mask]
# if 'level_0' in DATA.columns:
#     DATA = DATA.drop('level_0', axis=1)
# DATA.reset_index(inplace=True)
# # In[Compute some datas as instant displacement /speed / centering trajectories ...]


On trace le nombre de particules par frame en fonction du temps pour chaque manips. Ca permets de repérer des anomalies. 

In [ ]:
# Obtenir les expériences uniques pour les itérations
experiments = DATA['experiment'].unique()

# Déterminer le nombre de lignes et de colonnes pour les sous-graphiques
# Vous pouvez ajuster cela en fonction du nombre total d'expériences
n_cols = 2  # Nombre de colonnes, ajustez selon le besoin
n_rows = (len(experiments) + n_cols - 1) // n_cols  # Nombre de lignes

# Créer une figure et des axes pour les sous-graphiques
fig, axs = plt.subplots(n_rows, n_cols, figsize=(20 * n_cols, 10 * n_rows))
axs = axs.flatten()  # Aplatir le tableau d'axes si nécessaire

# Tracer les graphiques pour chaque expérience
for i, exp in enumerate(experiments):
    # Grouper les données par 'frame' et calculer le nombre de particules par frame
    nbr_part_per_frame = DATA[DATA['experiment'] == exp].groupby('time (min)')['particle'].nunique()

    # Tracer le graphique sur le sous-graphique correspondant
    ax = axs[i]
    ax.plot(nbr_part_per_frame.index, nbr_part_per_frame.values)
    ax.set_title(f'Nbr particle per Frame - {exp}')
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Number of particle')
    # ax.set_xlim([0, 340])
    # ax.set_ylim([0, 2000])  # Ajustez selon vos données

# Masquer les axes non utilisés s'il y en a
for ax in axs[len(experiments):]:
    ax.axis('off')

# Ajustement de la mise en page
plt.tight_layout()
# Ajustement de l'espacement et des marges
# plt.subplots_adjust(hspace=0.4, wspace=0.4)
# Enregistrer la figure entière
plt.savefig(f"{path_save_pic}Nbr_particle_per_Frame_manip_par_manip.jpg", format='jpg')

# Afficher la figure
plt.show()


In [ ]:
# Example usage
path_data = lib.calculate_total_path_first_frames(DATA, first_n_frames=100)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Grouper les données par 'experiment'
grouped = path_data.groupby('experiment')

# Calculer le nombre d'experiments pour déterminer le nombre de subplots nécessaires
n_experiments = len(grouped)

# Créer une figure et des axes pour les subplots
fig, axes = plt.subplots(nrows=n_experiments, figsize=(10, 4*n_experiments))

# Assurer que 'axes' est un array, même s'il n'y a qu'un seul subplot
if n_experiments == 1:
    axes = [axes]

for (experiment, group), ax in zip(grouped, axes):
    # Créer un histogramme pour chaque 'experiment'
    ax.hist(group['total_path_first_n'], bins=10, range=[0, 100], density=True, alpha=0.5)
    ax.set_title(f"Total path in first 100 frames for {experiment}")
    ax.set_xlabel('Lenght path (um)')
    ax.set_ylabel('Density')

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()


Calcul du nombre de cellules ayant un déplacement inférieur à une valeur.

In [ ]:
# Nombre de cellules avec un déplacement total inférieur à 10 sur les 20 premières frames
num_cells_low_displacement = path_data[path_data['total_path_first_n'] < 15]['particle'].nunique()

print("Nombre de cellules dont le déplacement est trop faible : " , num_cells_low_displacement)

In [ ]:
# # Get the indices of rows to drop
# to_drop = find_swaps_with_return(DATA)

# print(len(to_drop), " movements to delete there while the run is too much.")

# # # Drop the rows from DATA
# DATA = DATA.drop(to_drop)
DATA.reset_index(inplace=True)

Calcul des vitesses instantanées et des trajectoires recentrées

In [ ]:
# DATA = DATA[DATA['displacement [pix]'] < 5]
DATA = lib.center(traj=DATA)

print("\n"*2)
print(f"Le temps de lecture et de préparation des données pour la condition {CONDITION} est : ",
      (time.time() - INITIAL_TIME), 'min')
print("\n"*2)

In [ ]:
# %% [Compute the DATAS according to some parameters]
if ROLLING_MEAN:
    DATA = lib.rolling_mean(datas=DATA, roll=3)
if PIXELISATION:
    DATA = lib.pixelisation(datas=DATA, size_pix=SIZE_PIX)
if TIME_FRAME_STUDY:
    DATA, TIME_FRAME = lib.keep_nth_image(traj=DATA, n=N_FRAME, time_frame=TIME_FRAME)

In [ ]:
# #############################################################################
# %% [Calculation of total and cumulative displacement]
# #############################################################################
DATA, start_end = lib.length_displacement(traj=DATA, size_pix=SIZE_PIX)

In [ ]:
# # %% [Recalcul du max displacement]
# # ###################Erasing the suspicious displacements #####################
# grouped_data = DATA.groupby('particle')
# # Obtenir la valeur maximale de 'displacement' pour chaque groupe
# max_displacements = SIZE_PIX*grouped_data['displacement [pix]'].max()
# # Sélectionner les groupes dont la valeur maximale de 'displacement' est supérieure à 10
# selected_particles = max_displacements.loc[max_displacements > 50].index.tolist()
# bool_mask = DATA['particle'].isin(selected_particles)
# DATA_HIGH_DISP = DATA[bool_mask]
# if len(DATA_HIGH_DISP) > 0:
#     lib.plot_msd(msd=tp.imsd(traj=DATA_HIGH_DISP, mpp=SIZE_PIX, fps=FPS),
#                  fps=FPS, name='MSD with HIGHT DISP (sup at 10)', color_plot=COLOR_SUP,
#                  save=True, pathway_saving=path_save_pic, alpha=ALPHA, linewidth=LINEWIDTH)
# # Erasing the spurious traectories with too high displacement
# DATA = DATA[~bool_mask]

# #############################################################################
# #############################################################################
# We now Consider having all the good particles and all good datas.
# #############################################################################
# #############################################################################

In [ ]:
# %% [Plot all the trajectories]
fig, axis = plt.subplots(figsize=(10, 10))
# Assurer une échelle égale pour les axes
axis.set_aspect('equal', 'box')
plt.title('Trajectories after suspicious particles')
tp.plot_traj(DATA, label=(False))
plt.show()
fig.savefig(path_save_pic +
            'Trajectories after removing suspicious particles.jpg', format='jpg')

In [ ]:
# Construire le chemin complet
image_path = os.path.join(GENERAL_PATH_PICTURES, CONDITION_simple)
image_path = image_path + '_faits'
print(image_path)

In [ ]:
import trackpy as tp
import matplotlib.pyplot as plt

# Supposons que DATA est votre DataFrame
plot_exp = DATA.groupby('experiment')

# Déterminer le nombre de sous-graphiques basé sur le nombre d'expériences
num_experiments = len(plot_exp)
num_cols = 2  # Par exemple, vous pouvez définir 2 colonnes pour vos sous-graphiques
num_rows = (num_experiments + num_cols - 1) // num_cols  # Calculer le nombre de lignes nécessaire

# Créer la figure et les axes pour les sous-graphiques
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 10))  # Ajustez la taille selon vos besoins
axes = axes.flatten()  # Aplatir le tableau d'axes pour une itération facile

for ax, (exp_name, exp_data) in zip(axes, plot_exp):
    exp_directories = []
    for dirpath, dirnames, filenames in os.walk(image_path):
        for dirname in dirnames:
            if exp_name in dirname:
                full_path = os.path.join(dirpath, dirname)
                exp_directories.append(full_path)
    if exp_directories:
        image_path_directory = f'{exp_directories[0]}/mosaic/mosaic_total_0.tif'
        frame = imageio.imread(image_path_directory)
        ax.set_aspect('equal', 'box')
        ax.set_title(f'Trajectories after suspicious particles for {exp_name}')
        tp.plot_traj(exp_data, superimpose=frame, label=False, ax=ax)
    else:
        print(f"No directory found for {exp_name}")
        ax.axis('off')
# Ajuster la mise en page pour éviter le chevauchement
plt.tight_layout()
plt.show()
fig.savefig(path_save_pic + 'trajectories_on_frame_all_experiment.pdf', format='pdf')


In [ ]:
series_y = DATA.groupby('experiment')['y'].max().div(2048).apply(math.ceil).astype(int)
series_x = DATA.groupby('experiment')['x'].max().div(2048).apply(math.ceil).astype(int)
# Multiplication élément par élément entre les deux séries
result = series_y * series_x

In [ ]:
# %% [Mean speed]
# Grouper les données par 'frame' et calculer la moyenne de 'VitInst [um/min]'
mean_VitInst_per_frame = DATA.groupby('time (min)')['VitInst [um/min]'].mean()
mean_VitInst_per_frame.index = mean_VitInst_per_frame.index

mean_VitInst_per_frame = mean_VitInst_per_frame.rolling(5).mean().dropna()

lib.plot_datas(x_values=mean_VitInst_per_frame.index,
               y_values=mean_VitInst_per_frame.values,
               title='Mean VitInst [um/min] per Frame',
               x_label='time (min)', y_label='Mean VitInst [um/min]',
               x_lim=[0, max(mean_VitInst_per_frame.index)], y_lim=[0, 10], save=True,
               path_save_pic=path_save_pic, img_type="jpg")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Supposons que DATA est votre DataFrame
experiments = DATA['experiment'].unique()
n_experiments = len(experiments)

# Créer une figure avec deux colonnes de subplots pour chaque expérience
# La première colonne pour la vitesse instantanée moyenne par frame
# La deuxième colonne pour l'histogramme de la vitesse moyenne des particules
fig, axes = plt.subplots(n_experiments, 2, figsize=(20, 6*n_experiments), sharex='col', sharey='row')

# S'assurer que axes est toujours un array 2D pour faciliter l'itération
if n_experiments == 1:
    axes = np.expand_dims(axes, 0)

for i, exp in enumerate(experiments):
    # Filtrer les données pour l'expérience courante
    data_exp = DATA[DATA['experiment'] == exp]
    
    # Premier subplot : vitesse instantanée moyenne par frame
    mean_VitInst_per_frame_i = data_exp.groupby('time (min)')['VitInst [um/min]'].mean()
    mean_VitInst_per_frame_i.index = mean_VitInst_per_frame_i.index
    mean_VitInst_per_frame_smoothed_i = mean_VitInst_per_frame_i.rolling(5).mean().dropna()
    axes[i, 0].plot(mean_VitInst_per_frame_smoothed_i.index, mean_VitInst_per_frame_smoothed_i.values, label=f'Exp: {exp}')
    axes[i, 0].set_title(f'Mean VitInst [um/min] per Frame for {exp}')
    axes[i, 0].set_xlabel('time (min)')
    axes[i, 0].set_ylabel('Mean VitInst [um/min]')
    axes[i, 0].set_xlim([0, max(mean_VitInst_per_frame_i.index)])  # Ajustez selon vos données
    axes[i, 0].set_ylim([0, 15])   # Ajustez selon vos données
    axes[i, 0].legend()

    # Deuxième subplot : histogramme de la vitesse moyenne des particules pour l'expérience
    mean_VitInst_per_particle_i = data_exp.groupby('particle')['VitInst [um/min]'].mean()
    axes[i, 1].hist(mean_VitInst_per_particle_i, bins=30, alpha=0.5)
    axes[i, 1].set_title(f'Particle Mean VitInst [um/min] for {exp}')
    axes[i, 1].set_xlabel('Mean VitInst [um/min]')
    axes[i, 1].set_ylabel('Count')

# Ajuster automatiquement l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()

# Si vous souhaitez sauvegarder la figure entière
# plt.savefig(f"{path_save_pic}/combined_mean_vitinst_per_experiment.jpg", format="jpg")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supposons que DATA est un DataFrame Pandas avec vos données
experiences_uniques = DATA['experiment'].unique()
nombre_experiences = len(experiences_uniques)

# Créer une grille de subplots avec un nombre approprié de lignes
fig, axs = plt.subplots(nombre_experiences, 1, figsize=(10, 5 * nombre_experiences), sharex=True)

# Générer une palette de couleurs
palette = plt.cm.get_cmap('tab10', nombre_experiences)  # 'tab10' est une palette de 10 couleurs

# Définir les limites de l'axe des x basées sur les données globales
x_min = DATA.groupby('particle')['VitInst [um/min]'].mean().min()
x_max = DATA.groupby('particle')['VitInst [um/min]'].mean().max()

# Assurez-vous que axs est un array, même s'il n'y a qu'un seul subplot
if nombre_experiences == 1:
    axs = [axs]

# Remplir chaque subplot
for idx, exp in enumerate(experiences_uniques):
    # Calculer la moyenne de la vitesse instantanée pour chaque particule
    moyennes_vitesses = DATA[DATA['experiment'] == exp].groupby('particle')['VitInst [um/min]'].mean()
    
    # Créer un histogramme sur le subplot correspondant avec une couleur unique
    axs[idx].hist(moyennes_vitesses, bins=30, alpha=0.3, color=palette(idx))
    axs[idx].set_title(f'Expérience: {exp}')
    axs[idx].set_xlabel('Vitesse Instantanée (µm/min)')
    axs[idx].set_ylabel('Fréquence')
    axs[idx].set_xlim([x_min, x_max])  # Appliquer la même échelle des x à tous les subplots

# Ajuster l'espace entre les subplots pour éviter le chevauchement
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Obtenir la liste unique des expériences
experiments = DATA['experiment'].unique()
n_experiments = len(experiments)

# Créer une figure et un ensemble de subplots
# Ajustez nrows et ncols selon le nombre d'experiences que vous avez
fig, axes = plt.subplots(nrows=n_experiments, ncols=1, figsize=(10, 5*n_experiments))

# S'assurer que 'axes' est un array pour faciliter l'itération, même s'il n'y a qu'une seule expérience
if n_experiments == 1:
    axes = [axes]

for ax, exp in zip(axes, experiments):
    # Sélectionner les données pour l'expérience courante
    data_exp = DATA[DATA['experiment'] == exp]
    
    # Calculer la vitesse instantanée moyenne pour chaque particule
    mean_vitinst_per_particle = data_exp.groupby('particle')['VitInst [um/min]'].mean()
    
    # Tracer l'histogramme sur le subplot correspondant
    ax.hist(mean_vitinst_per_particle, bins=30, alpha=0.3)
    ax.set_title(f'Vitesse instantanée moyenne [um/min] - {exp}')
    ax.set_xlabel('VitInst [um/min]')
    ax.set_ylabel('Nombre de particules')

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculer les limites globales pour la vitesse instantanée moyenne
all_mean_vitinst = DATA.groupby(['experiment', 'particle'])['VitInst [um/min]'].mean()
global_min, global_max = all_mean_vitinst.min(), all_mean_vitinst.max()

# Ajuster légèrement les limites pour une meilleure visualisation
global_min, global_max = global_min - (global_max - global_min) * 0.1, global_max + (global_max - global_min) * 0.1

# Obtenir la liste unique des expériences
experiments = DATA['experiment'].unique()
n_experiments = len(experiments)

# Créer une figure et un ensemble de subplots
fig, axes = plt.subplots(nrows=n_experiments, ncols=1, figsize=(10, 5*n_experiments), sharex=True, sharey=True)

# S'assurer que 'axes' est un array pour faciliter l'itération, même s'il n'y a qu'une seule expérience
if n_experiments == 1:
    axes = [axes]

for ax, exp in zip(axes, experiments):
    # Sélectionner les données pour l'expérience courante
    data_exp = DATA[DATA['experiment'] == exp]
    
    # Calculer la vitesse instantanée moyenne pour chaque particule
    mean_vitinst_per_particle = data_exp.groupby('particle')['VitInst [um/min]'].mean()
    
    # Tracer l'histogramme sur le subplot correspondant
    ax.hist(mean_vitinst_per_particle, bins=30, alpha=0.3, range=(global_min, global_max))
    ax.set_title(f'Vitesse instantanée moyenne [um/min] - {exp}')
    ax.set_xlabel('VitInst [um/min]')
    ax.set_ylabel('Nombre de particules')
    
    # Calculer et tracer la médiane en rouge
    median_value = mean_vitinst_per_particle.median()
    ax.axvline(median_value, color='red', linestyle='dashed', linewidth=1)
    ax.text(median_value, ax.get_ylim()[1]*0.95, f'Median: {median_value:.2f}', color='red', ha='right')

    # Appliquer les mêmes limites d'axes à tous les subplots
    ax.set_xlim(global_min, global_max)

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()


In [ ]:
# %% [Number of particle on each frame]

# Grouper les données par 'frame' et calculer la moyenne de 'VitInst [um/min]'
nbr_part_per_frame = DATA.groupby('time (min)')['particle'].nunique()

lib.plot_datas(x_values=nbr_part_per_frame.index, y_values=nbr_part_per_frame.values,
               title='Nbr particle per Frame',
               x_label='time (min)', y_label='Number of particle',
               x_lim=[0, max(nbr_part_per_frame.index)], y_lim=[0, 1000], save=True,
               path_save_pic=path_save_pic, img_type="jpg")

In [ ]:
import pandas as pd
import numpy as np

# Supposons que df est votre DataFrame contenant les résultats du tracking
# df devrait avoir des colonnes 'particle', 'frame', 'x', et 'y'
df = DATA
# Fonction pour calculer le vecteur de déplacement
def displacement_vectors(df):
    df['dx'] = df.groupby('particle')['x'].diff()
    df['dy'] = df.groupby('particle')['y'].diff()
    return df.dropna()

# Normaliser les vecteurs de déplacement
def normalize_vectors(df):
    df = df.copy()
    magnitude = np.sqrt(df['dx']**2 + df['dy']**2)
    df.loc[:, 'dx_norm'] = df['dx'] / magnitude
    df.loc[:, 'dy_norm'] = df['dy'] / magnitude
    return df

# Calculer l'angle des vecteurs
# Arctan2 permet de calculer l'angle en radians entre la partie positive de l'axe des abscisses d'un plan et le point (x,y)
# Angle positif dans le sens trigo et négativ dans le sens inverse trigo
def calculate_angles(df):
    df = df.copy()
    df.loc[:, 'angle'] = np.arctan2(df['dy_norm'], df['dx_norm'])
    return df

# Calculer l'autocorrélation directionnelle
def direction_autocorrelation(df, max_lag):
    results = []
    for particle in df['particle'].unique():
        particle_df = df[df['particle'] == particle].copy()
        for lag in range(1, max_lag + 1):
            particle_df['angle_lag'] = particle_df['angle'].shift(-lag)
            cos_diff = np.cos(particle_df['angle'] - particle_df['angle_lag'])
            autocorr = cos_diff.mean()
            results.append({'particle': particle, 'lag': lag, 'autocorrelation': autocorr})
    return pd.DataFrame(results).dropna()

# Appliquer les fonctions
df = displacement_vectors(df)
df = normalize_vectors(df)
df = calculate_angles(df)
max_lag = 10  # Ajustez en fonction de la longueur de vos trajectoires
autocorr_df = direction_autocorrelation(df, max_lag)

In [ ]:
DATA2 = DATA
DATA2['frame'] = pd.factorize(DATA2['frame'])[0]
IMSD = tp.imsd(traj=DATA2,
               mpp=SIZE_PIX, fps=FPS,
               max_lagtime=200, statistic='msd',
               pos_columns=None)
IMSD

In [ ]:
lib.plot_msd(IMSD, fps=FPS, name="MSD of all frames in function of lag time (s)",
             color_plot = 'red', save=False, pathway_saving=None,
             alpha=0.5, linewidth=0.3, img_type='jpg')

In [ ]:
# %% [traj clustering with fit and defining a cutoff]
LAG_TIME_FIT = 5
# Compute et plot the director factor of the imsd
importlib.reload(lib)

COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF =\
    lib.traj_clustering_with_fit_cutoff(DATA2, imsd=IMSD, hist=True,
                                        lag_time_fit=LAG_TIME_FIT,
                                        micronperpixel=SIZE_PIX,
                                        fps=FPS, binsize=250,
                                        peak_height=50, peak_width=1,
                                        save=True, pathway_fig=path_save_pic,
                                        name='all the experiment autocorr', img_type="jpg",
                                        plot=True, color_sup_inf=color_sup_inf,
                                        cutoff_default=0.5
                                        )

# # DATA_INF, DATA_SUP, IMSD_INF, IMSD_SUP,
# DATA_INF = DATA[DATA['particle'].isin(PART_COEF_INF)]
# DATA_SUP = DATA[DATA['particle'].isin(PART_COEF_SUP)]
# IMSD_INF = IMSD.loc[:, IMSD.columns.isin(PART_COEF_INF)]
# IMSD_SUP = IMSD.loc[:, IMSD.columns.isin(PART_COEF_SUP)]

In [ ]:
# for numero_particule in df_negative['particle'].unique():
#     print(numero_particule)
#     print("Etude de la particule : ", numero_particule, ' dans la manip', DATA[DATA['particle']==numero_particule]['experiment'].iloc[0])
#     lib.create_cropped_tracking_gif(datas=DATA, target_particle = numero_particule,
#                                     condition = CONDITION_simple,
#                                     dot_size= 7,
#                                     crop_size=100, 
#                                     gif=False,
#                                     pathway_saving=None,
                                    # pathway_initial='/Volumes/Labo_Alex_Mac/A_analyser/')

In [ ]:
# Assurez-vous que sums_df est un DataFrame Pandas
sums_df = pd.DataFrame(columns=['experiment', 'particle', 'displacement_sum'])

rows = []

for exp in DATA['experiment'].unique():
    exp_data = DATA[DATA['experiment'] == exp]
    for particle_id, part in exp_data.groupby('particle'):
        displacement_sum = part['displacement [pix]'].head(200).sum()
        new_row = {'experiment': exp, 'particle': particle_id, 'displacement_sum': displacement_sum}
        rows.append(new_row)

# Création d'un nouveau DataFrame à partir de la liste de dictionnaires
sums_df = pd.DataFrame(rows)


In [ ]:
mean_sum = []
for _, exp in sums_df.groupby('experiment'):
    mean_sum.append(exp['displacement_sum'].mean())

plt.hist(mean_sum, bins = 100)
plt.title("Nbr of exp in function of mean displacement per particule")
plt.savefig(path_save_pic + f"Nbr of exp in function of mean displacement per particule {CONDITION_simple}.png", format='png') 
plt.show()

In [ ]:
# Y'a t'il l'air d'enregistrer? 
# size en fonction ? 
# faire des manip en x10
# calculer un pseudo packing fraction : nombre de cellule par unité d'air. Ensuite, on essaye de voir la size des cells. 
# Cela dit, c'est vraiment le nombre de cellules par unité d'aire qui m'interesse

In [ ]:
importlib.reload(lib)
lib.plot_displacement(DATA, start_end=start_end, alpha = 0.5, linewidth=0.5, ylim=[0, 750], xlim=[0, max(DATA['time (min)'])])
plt.show()

In [ ]:
plt.hist(start_end, bins=250)
plt.xlim([0,400])

In [ ]:
for exp in DATA['experiment'].unique():
    # Calculer ymax et xmax pour chaque expérience
    ymax = math.ceil(DATA[DATA['experiment'] == exp]['y'].max() / 2048)
    xmax = math.ceil(DATA[DATA['experiment'] == exp]['x'].max() / 2048)

    # Convertir en int si nécessaire (math.ceil retourne déjà un int)
    ymax = int(ymax)
    xmax = int(xmax)
    # Filtrer le DataFrame pour l'expérience et les 200 premières frames
    exp_data = DATA[(DATA['experiment'] == exp) & (DATA['frame'] < 340)]
    nbr_particles = exp_data['particle'].nunique()
 
    # # Grouper par 'frame' et compter les particules
    # particules_par_frame = exp_data.groupby('frame')['particle'].nunique()

    # # Calculer la moyenne du nombre de particules
    # moyenne_particules = particules_par_frame.mean()
    # nombre_part_par_champs = moyenne_particules/(xmax*ymax)
    nombre_part_par_champs = nbr_particles/(xmax*ymax)
    print(f"Nombre de cellules par champs pour la manips {exp}:", nombre_part_par_champs)

In [ ]:
import pandas as pd
import math


# Liste pour stocker les données de chaque expérience
data = []

for exp in DATA['experiment'].unique():
    # Calculer ymax et xmax
    ymax = math.ceil(DATA[DATA['experiment'] == exp]['y'].max() / 2048)
    xmax = math.ceil(DATA[DATA['experiment'] == exp]['x'].max() / 2048)
    ymax, xmax = int(ymax), int(xmax)

    # Filtrer pour l'expérience et les 200 premières frames
    exp_data = DATA[(DATA['experiment'] == exp) & (DATA['frame'] < 200)]

    # Compter les particules par frame et calculer la moyenne
    moyenne_particules = exp_data.groupby('frame')['particle'].nunique().mean()
    nombre_part_par_champs = int(moyenne_particules / (xmax * ymax))

    # Calculer mean_sum pour l'expérience
    mean_sum = int(sums_df[sums_df['experiment'] == exp]['displacement_sum'].mean())

    #Calcul de la vitesse moyenne des particules
    mean_speed = exp_data.groupby('particle')['VitInst [um/min]'].mean().mean()


    # Ajouter les données calculées à la liste
    data.append({
        'experiment': exp,
        'mean_sum': mean_sum,
        'taille': xmax * ymax,
        'nombre_part_par_champs': nombre_part_par_champs,
        'mean_speed [um/min]': mean_speed,
    })

# Création du DataFrame
result_df = pd.DataFrame(data)

# # Affichage du DataFrame
# print(result_df)

In [ ]:
import matplotlib.pyplot as plt

# Supposons que result_df est votre DataFrame contenant les données nécessaires

# Création d'une grille de sous-graphiques 2x2
fig, axes = plt.subplots(1, 2, figsize=(20, 6))  # Ajustez la taille selon vos besoins

# Aplatir le tableau d'axes pour un accès plus facile
ax1, ax2, = axes.flatten()

# Premier graphique: Mean Sum vs. Nombre de Particules par Champ
ax1.scatter(result_df['nombre_part_par_champs'], result_df['mean_sum'], marker='+', color='orange')
ax1.set_title('Average distance traveled vs. Number of particles per field')
ax1.set_xlabel('Number of particles per field')
ax1.set_ylabel('Average distance traveled')

# Deuxième graphique: Mean Speed vs. Nombre de Particules par Champ
ax2.scatter(result_df['nombre_part_par_champs'], result_df['mean_speed [um/min]'], marker='+', color='blue')
ax2.set_title('Mean Speed vs. Number of particles per field')
ax2.set_xlabel('Number of particles per field')
ax2.set_ylabel('Mean Speed [um/min]')

# # Masquer le quatrième axe car il n'est pas utilisé
# ax4.axis('off')

# Ajustement de la mise en page pour éviter le chevauchement des titres
plt.tight_layout()

# Afficher les graphiques
plt.show()

fig.savefig(path_save_pic + f"graph in function of nbr of particles per field {CONDITION_simple}.png", format='png') 

In [ ]:
path=filtered_DATA[filtered_DATA['particle']==filtered_DATA['particle'].unique()[0]]['experiment'].iloc[0]
import glob
glob.glob(f'/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x_faits/*{path}*')
path

In [ ]:
# importlib.reload(lib)
# import glob
# for i in filtered_DATA['particle'].unique():
#     path=filtered_DATA[filtered_DATA['particle']==i]['experiment'].iloc[0]
#     path_frames = glob.glob(f'/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x/faits/*{path}*')[0]
#     print('path frames' , path_frames)
#     lib.create_cropped_tracking_gif(datas=filtered_DATA, target_particle=i,
#                                     condition=None, crop_size = 200, 
#                                     dot_size = 15, gif = False,
#                                     pathway_saving = '/Users/souchaud/Desktop/gif',
#                                     pathway_initial = path_frames + '/mosaic/')

In [ ]:
# Récupérer les particules dont le coefficient inférieur est inférieur à 0.2
particules_filtrées = [part for coef, part in zip(COEF_INF, PART_COEF_INF) if coef < 0.25]
# regarder de quelle manip
DATA_INF_025 = DATA[DATA['particle'].isin(particules_filtrées)]
DATA_INF_025_136 = DATA_INF_025[DATA_INF_025['experiment']=='ASMOT136']
fig, axis = plt.subplots(figsize=(10, 10))
# Assurer une échelle égale pour les axes
axis.set_aspect('equal', 'box')
plt.title('Trajectories after suspicious particles')
tp.plot_traj(DATA_INF_025_136, label=(False))
plt.show()

In [ ]:

# Pour afficher entièrement le DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Affichez par exemple les premières ou dernières N lignes
DATA_INF_025_136.head(10)
# DATA_INF_025.groupby('experiment')['particle'].nunique()

In [ ]:
DATA_INF_025_136['frame'].unique()

In [ ]:
# Lister tous les éléments dans le chemin racine
chemin_racine = '/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x_faits/'
for nom in os.listdir(chemin_racine):
    # Construire le chemin complet de l'élément
    chemin_complet = os.path.join(chemin_racine, nom)
    # Vérifier si c'est un dossier et si "ASMOT136" est dans le nom
    if os.path.isdir(os.path.join(chemin_racine, nom)) and "ASMOT136" in nom:
        chemin_final = chemin_complet
        break


In [ ]:
dossiers_asmot136 = [nom for nom in os.listdir(chemin_racine)
                     if os.path.isdir(os.path.join(chemin_racine, nom)) and "ASMOT136" in nom]

for dossier in dossiers_asmot136:
    print(dossier)

In [ ]:
lib.create_cropped_tracking_gif(datas=DATA_INF_025, target_particle = None,
                                condition: str = None, crop_size: int = None, 
                                dot_size: int = 25, gif: bool = False,
                                pathway_saving=None,
                                pathway_initial=chemin_complet + '/mosaic/')

In [ ]:
from PIL import Image, ImageDraw
import os
import pandas as pd
import glob


def create_cropped_tracking_gif(datas, condition=None, dot_size=15, gif=True, pathway_saving=None, pathway_initial=None):
    """Creates a cropped tracking GIF from particle data.

    Args:
        datas (pd.DataFrame): DataFrame containing particle data with columns 'frame', 'x', and 'y'.
        condition (str, optional): Optional condition to filter files within the initial pathway. Defaults to None.
        dot_size (int, optional): Size of the dots representing particles. Defaults to 15.
        gif (bool, optional): Whether to create a GIF (True) or individual PNG frames (False). Defaults to True.
        pathway_saving (str, optional): Path to save the GIF or PNG frames. Defaults to None (automatic creation).
        pathway_initial (str, optional): Path to the directory containing image frames. Defaults to None.

    Raises:
        ValueError: If `datas` is not a pandas DataFrame.
    """

    if not isinstance(datas, pd.DataFrame):
        raise ValueError("Data must be a pandas DataFrame.")

    if pathway_initial is None:
        pathway_initial = '/Users/souchaud/Desktop/A_analyser/'
    dossier_manip = glob.glob(f'{pathway_initial}{condition}/*') if condition else glob.glob(f'{pathway_initial}*')
    if not dossier_manip:
        print("No such file")
        return
    pathway_experiment = dossier_manip[0] + '/mosaic/' if condition else pathway_initial

    if pathway_saving is None:
        path = '/users/souchaud/Desktop/Analyses/gif_particle_seule/'
        os.makedirs(path, exist_ok=True)
        condition_path = f"{condition}_" if condition else ""
        pathway_saving = os.path.join(path, f'{condition_path}all_particles/')
    os.makedirs(pathway_saving, exist_ok=True)

    # Clear existing files (can be optimized based on deletion preference)
    for filename in os.listdir(pathway_saving):
        file_path = os.path.join(pathway_saving, filename)
        os.remove(file_path)

    frames = []  # List to store image frames for GIF creation

    for frame_number in range(datas['frame'].min(), datas['frame'].max() + 1):
        image_path = os.path.join(pathway_experiment, f"mosaic_total_{frame_number}.tif")
        with Image.open(image_path) as img:
            # Convert to RGBA for transparency (optional, comment out if not needed)
            # img = img.convert("RGBA")

            draw = ImageDraw.Draw(img)

            frame_data = datas[datas['frame'] == frame_number]

            for _, particle in frame_data.iterrows():
                red_transparent = (255, 0, 0, 127)  # Rouge semi-transparent
                draw.ellipse([(particle['x'] - dot_size / 2, particle['y'] - dot_size / 2),
                              (particle['x'] + dot_size / 2, particle['y'] + dot_size / 2)],
                              fill=red_transparent)

            if gif:
                # Efficiently append to frames list for GIF creation
                frames.append(img.copy())
            else:
                output_path = os.path.join(pathway_saving, f"frame_{frame_number}.png")
                img.save(output_path, format="PNG")  # Adjust quality parameter for smaller PNGs

    if gif:
        # Create GIF using an appropriate library (e.g., moviepy, PillowGif)
        # ... (implementation using your preferred library)
        # Example using moviepy (install with `pip install moviepy`):
        from moviepy.editor import ImageSequenceClip
        clip = ImageSequenceClip(frames, fps=10)  # Adjust fps as needed
        clip.write_gif(os.path.join(pathway_saving, "tracking.gif"), quantizer=8)  # Lower quantizer for smaller GIFs



In [ ]:
from PIL import Image, ImageDraw
import os
import pandas as pd
import glob
import shutil

def create_cropped_tracking_gif(datas, condition=None, dot_size=15, gif=False, pathway_saving=None, pathway_initial=None):
    if not isinstance(datas, pd.DataFrame):
        raise ValueError("Data must be a pandas DataFrame.")

    if pathway_initial is None:
        pathway_initial = '/Users/souchaud/Desktop/A_analyser/'
    dossier_manip = glob.glob(f'{pathway_initial}{condition}/*') if condition else glob.glob(f'{pathway_initial}*')
    if not dossier_manip:
        print("No such file")
        return
    pathway_experiment = dossier_manip[0] + '/mosaic/' if condition else pathway_initial

    if pathway_saving is None:
        path = '/users/souchaud/Desktop/Analyses/gif_particle_seule/'
        os.makedirs(path, exist_ok=True)
        condition_path = f"{condition}_" if condition else ""
        pathway_saving = os.path.join(path, f'{condition_path}all_particles/')
    os.makedirs(pathway_saving, exist_ok=True)

    for filename in os.listdir(pathway_saving):
        file_path = os.path.join(pathway_saving, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

    # Tracer toutes les particules pour chaque frame
    for frame_number in range(datas['frame'].min(), datas['frame'].max() + 1):
        image_path = os.path.join(pathway_experiment, f"mosaic_total_{frame_number}.tif")
        with Image.open(image_path) as img:
            # Convertir en RGBA pour supporter la transparence
            img = img.convert("RGBA")
            draw = ImageDraw.Draw(img)

            frame_data = datas[datas['frame'] == frame_number]
            for _, particle in frame_data.iterrows():
                red_transparent = (255, 0, 0, 100)  # Rouge semi-transparent
                draw.ellipse([(particle['x'] - dot_size / 2, particle['y'] - dot_size / 2),
                              (particle['x'] + dot_size / 2, particle['y'] + dot_size / 2)],
                              fill=red_transparent)

            output_path = os.path.join(pathway_saving, f"frame_{frame_number}.png")
            img.save(output_path, format="PNG")

    # Si gif est demandé, cette partie nécessite une adaptation pour gérer tous les frames sans se focaliser sur une particule spécifique
    # Il faut collecter tous les images sauvegardées et les assembler en un GIF
    # Cela peut être implémenté en fonction de vos besoins spécifiques

# Exemple d'utilisation
# Assurez-vous que DATA_INF_025 est défini et que le chemin_initial pointe vers le dossier correct
create_cropped_tracking_gif(datas=DATA_INF_025, condition=None, dot_size=25,
                            gif=False, pathway_saving='/Users/souchaud/Desktop/A_analyser/test2/',
                            pathway_initial=chemin_final + '/mosaic/')
